In [1]:
!pip install kafka-python
!pip install psycopg2-binary

In [21]:
import psycopg2
from kafka import KafkaConsumer, TopicPartition
from json import loads
import time
from pyspark.sql import SparkSession
from multiprocessing import Process, Manager
import json

In [22]:
consumer = KafkaConsumer('ny_tripdata',
                         bootstrap_servers=['nyc_tripdata_kafka_1:29092'],
                         auto_offset_reset='earliest',
                         group_id='group')

In [7]:
res = consumer.poll(max_records=6)

In [24]:
for k in res:
    print(k)

TopicPartition(topic='ny_tripdata', partition=0)


In [27]:
res[topic][0].value

b'{"vendor_name":"VTS","Trip_Pickup_DateTime":"2009-01-25 00:46:00","Trip_Dropoff_DateTime":"2009-01-25 00:59:00","Passenger_Count":"1","Trip_Distance":"3.1899999999999999","Start_Lon":"-73.968412000000001","Start_Lat":"40.762372999999997","Rate_Code":null,"store_and_forward":null,"End_Lon":"-74.002967999999996","End_Lat":"40.740091999999997","Payment_Type":"CASH","Fare_Amt":"10.9","surcharge":"0.5","mta_tax":null,"Tip_Amt":"0","Tolls_Amt":"0","Total_Amt":"11.4"}'

In [29]:
data = []
for message in res[topic]:
    msg  = json.loads(message.value.decode("utf-8"))
    tmp =  get_most_relevant_feature(msg)
    tmp = json.dumps(tmp)
    data.append(tmp)

In [30]:
data

['{"vendor_name": "VTS", "Trip_Pickup_DateTime": "2009-01-25 00:46:00", "Trip_Dropoff_DateTime": "2009-01-25 00:59:00", "Trip_Distance": "3.1899999999999999", "Start_Lon": "-73.968412000000001", "Start_Lat": "40.762372999999997", "End_Lon": "-74.002967999999996", "End_Lat": "40.740091999999997"}',
 '{"vendor_name": "VTS", "Trip_Pickup_DateTime": "2009-01-25 12:16:00", "Trip_Dropoff_DateTime": "2009-01-25 12:33:00", "Trip_Distance": "2.2999999999999998", "Start_Lon": "-73.994609999999994", "Start_Lat": "40.750591999999997", "End_Lon": "-73.993575000000007", "End_Lat": "40.733224999999997"}',
 '{"vendor_name": "VTS", "Trip_Pickup_DateTime": "2009-01-25 19:49:00", "Trip_Dropoff_DateTime": "2009-01-25 20:22:00", "Trip_Distance": "19.210000000000001", "Start_Lon": "-73.783422999999999", "Start_Lat": "40.648646999999997", "End_Lon": "-74.00658", "End_Lat": "40.705803000000003"}',
 '{"vendor_name": "CMT", "Trip_Pickup_DateTime": "2009-01-06 02:06:20", "Trip_Dropoff_DateTime": "2009-01-06 02:0

In [6]:
manager = Manager()
data_queue = manager.Queue()
data_queue.qsize()

0

In [7]:
data_queue.get()

KeyboardInterrupt: 

In [28]:
def get_most_relevant_feature(msg):
    data = {}
    data['vendor_name'] = msg['vendor_name']
    data['Trip_Pickup_DateTime'] = msg['Trip_Pickup_DateTime']
    data['Trip_Dropoff_DateTime'] = msg['Trip_Dropoff_DateTime']
    data['Trip_Distance'] = msg['Trip_Distance']
    data['Start_Lon'] = msg['Start_Lon']
    data['Start_Lat'] = msg['Start_Lat']
    data['End_Lon'] = msg['End_Lon']
    data['End_Lat'] = msg['End_Lat']
    return data

In [6]:
def insert_trip(conn,
                cur,
                vendor_name, 
                Trip_Pickup_DateTime, 
                Trip_Dropoff_DateTime,
                Trip_Distance,
                Start_Lon,
                Start_Lat,
                End_Lon,
                End_Lat):

    sql = """INSERT INTO trips (vendor_name, 
                                Trip_Pickup_DateTime, 
                                Trip_Dropoff_DateTime, 
                                Trip_Distance, 
                                Start_Lon, 
                                Start_Lat, 
                                End_Lon, 
                                End_Lat)
             VALUES(%s,%s,%s,%s,%s,%s,%s,%s);"""
    
    # execute the INSERT statement
    cur.execute(sql, (vendor_name, 
                      Trip_Pickup_DateTime, 
                      Trip_Dropoff_DateTime,
                      Trip_Distance,
                      Start_Lon,
                      Start_Lat,
                      End_Lon,
                      End_Lat,))
    # get the generated id back
    # vendor_id = cur.fetchone()[0]
    # commit the changes to the database
    conn.commit()

In [7]:
def consumeData(consumer, data_queue):
    try:
        # connect to the PostgreSQL database
        conn = psycopg2.connect(host="docker-compose_db_1",
                                database="postgres",
                                user="postgres",
                                password="postgres1234")
        # create a new cursor
        cur = conn.cursor()
        
        for message in consumer:
            try:
                msg = json.loads(message.value.decode("utf-8"))
                data = get_most_relevant_feature(msg)
                insert_trip(conn, cur, *list(data.values()))
                data_queue.put(data)
                print("Progress: {}".format(data_queue.qsize()), end="\r", flush=True)
            except Exception as e:
                print(data)
                print(e)
                pass
        # close communication with the database
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

t1 = Process(target=consumeData, args=(consumer, data_queue, ))

In [8]:
t1.start()

In [12]:
data_queue.qsize()

998

In [13]:
t1.terminate()
del t1

In [ ]:
plt.figure(figsize = (10,10))
plt.plot(longitude,latitude,'.', alpha = 0.4, markersize = 0.05)
plt.show()